In [31]:
import os 
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader 
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_openai.embeddings import OpenAIEmbeddings
load_dotenv()


True

In [32]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
Model = "gpt-3.5-turbo"
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=Model)
embeddings = OpenAIEmbeddings()
parser = StrOutputParser()
chain = model | parser
chain.invoke("tell me a joke")


"Why don't scientists trust atoms?\n\nBecause they make up everything!"

In [33]:
loader = PyPDFLoader("IEEEComparativeStudy.pdf")
pages = loader.load_and_split()

In [37]:

template = """
You are professioanl in fraud detection as ML engineer.
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".
Context: {context}
Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

"I don't know"

In [34]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [36]:
retriever = vectorstore.as_retriever()
retriever.invoke("why Radom  Forest is better ? ")

[Document(metadata={'source': 'IEEEComparativeStudy.pdf', 'page': 3}, page_content='distribution within the folds . Meanwhile, to fill the number of \nthe negative files, we proceed to the re -balancing of the data \nthrough the method "Resample" of Weka before applying \ntesting and learning  in each iteration, the results obtained after \nre-balancing data are improved.  \n5) Evaluation  \n \nThe efficiency and performance of the algorithms are \ncalculated from various metrics , the most prominent are recall \n(sensitivity ), specificity, and precision. An efficient classifier \nshould have higher values for these indicators.  To facilitate the \ninterpretation  of the algorithm performance, Van Rijsberjen, \n(1979)  [21] has created a synthetic measure  F1-Measure  or F-\nscore , defined as  the harmonic mean of the precision ( True \nPositive Accuracy ) and recall  (True Positive Rate)  of a binary \ndecision rule . The F -measure recognized as the most \ncommonly used measure to 

In [39]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)
chain.invoke("why Radom  Forest is better ? ")

TypeError: string indices must be integers